#### Convolutional Neural Network

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5 # This sets the maximum allowed GPU utilization to 50%
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

No nedd to reun the above code if one has not enabled the tensorflow GPU

In [1]:
# Convolutional Neural Network

# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.4.1'

In [4]:
# Part 1 - Data Preprocessing

# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)


In [5]:
training_set = train_datagen.flow_from_directory('Data/DogCat/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('Data/DogCat/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8003 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [6]:
from tensorflow.keras.layers import Conv2D

In [7]:
# Part 2 - Building the CNN

# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32,padding="same",kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [8]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1048704   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [9]:
# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the CNN on the Training set and evaluating it on the Test set
cnn.fit(x = training_set, validation_data = test_set, epochs = 20)

Epoch 1/20
251/251 [==============================] - 43s 167ms/step - loss: 0.7124 - accuracy: 0.5129 - val_loss: 0.6274 - val_accuracy: 0.6599
Epoch 2/20
251/251 [==============================] - 42s 166ms/step - loss: 0.6177 - accuracy: 0.6682 - val_loss: 0.5692 - val_accuracy: 0.7059
Epoch 3/20
251/251 [==============================] - 42s 166ms/step - loss: 0.5728 - accuracy: 0.7083 - val_loss: 0.5395 - val_accuracy: 0.7380
Epoch 4/20
251/251 [==============================] - 42s 167ms/step - loss: 0.5344 - accuracy: 0.7371 - val_loss: 0.5207 - val_accuracy: 0.7439
Epoch 5/20
251/251 [==============================] - 38s 151ms/step - loss: 0.5080 - accuracy: 0.7484 - val_loss: 0.5184 - val_accuracy: 0.7405
Epoch 6/20
251/251 [==============================] - 39s 154ms/step - loss: 0.4854 - accuracy: 0.7637 - val_loss: 0.4747 - val_accuracy: 0.7726
Epoch 7/20
251/251 [==============================] - 37s 149ms/step - loss: 0.4661 - accuracy: 0.7685 - val_loss: 0.4626 - val_ac

In [10]:
# save it as a h5 file


from tensorflow.keras.models import load_model

cnn.save('saved_models/DogCats CNN/model_cat_dog.h5')

In [11]:
from tensorflow.keras.models import load_model
 
# load model
model = load_model('saved_models/DogCats CNN/model_cat_dog.h5')

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1048704   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [27]:
# Part 4 - Making a single prediction

import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('Data/DogCat/test_set/dogs/dog.4002.jpg', target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)

In [28]:
result

array([[0.7282966]], dtype=float32)

In [31]:
# Part 4 - Making a single prediction

import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('Data/DogCat/test_set/cats/cat.4012.jpg', target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)

In [32]:
result

array([[0.11662069]], dtype=float32)

In [33]:
if result[0]<=0.5:
    print("The image classified is cat")
else:
    print("The image classified is dog")

The image classified is cat
